In [9]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd



import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from tensorflow import keras
from operations import adjust_dataset, split_data
from keras.layers import Dropout



# Implementation of ANN

In [10]:

df = pd.read_csv('docs/data/GlobalWeatherRepository.csv')

df = adjust_dataset(df, 'Madrid')


In [11]:
possibilities = ['temperature_celsius', 'wind_kph', 'humidity', 'pressure_mb', 'precip_mm', 'cloud']


X_train, y_train, X_test, y_test = split_data(df, 'temperature_celsius', int(len(df) * 0.7))

# print(X_train.shape)

In [15]:
# Lets build the model
# We will use a simple neural network with 3 layers and a sliding window of 7 days and 3 days as output


# We will use a sliding window of 7 days and 3 days as output
window_size = 7
output_size = 3

def create_sequences_and_labels(data, window_size, output_size):
    sequences, labels = [], []
    for i in range(len(data) - window_size - output_size + 1):
        seq = data[i:i + window_size]
        label = data[i + window_size:i + window_size + output_size]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

# Create sequences and labels for training data
X_train_seq, y_train_seq = create_sequences_and_labels(y_train, window_size, output_size)

# Create sequences and labels for test data
X_test_seq, y_test_seq = create_sequences_and_labels(y_test, window_size, output_size)

# Define the model
model = keras.Sequential([
    keras.layers.Dense(64, input_shape=(window_size,), activation='relu'),
    Dropout(0.2),   # Dropout layer to prevent overfitting, getting rid of 20% of the neurons
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(output_size, activation='linear')
])

# Compilation of model
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mean_squared_error'])

# Train the model
model.fit(X_train_seq, y_train_seq, epochs=100)

# Evaluate the model on test data
test_loss, test_mse = model.evaluate(X_test_seq, y_test_seq, verbose=2)

rmse = np.sqrt(test_mse)
print('mean squared error:',np.round(rmse, 2))


Epoch 1/100
2/2 [==============================] - 1s 6ms/step - loss: 224.9698 - mean_squared_error: 224.9698
Epoch 2/100
2/2 [==============================] - 0s 7ms/step - loss: 155.5032 - mean_squared_error: 155.5032
Epoch 3/100
2/2 [==============================] - 0s 6ms/step - loss: 96.8974 - mean_squared_error: 96.8974
Epoch 4/100
2/2 [==============================] - 0s 8ms/step - loss: 58.0259 - mean_squared_error: 58.0259
Epoch 5/100
2/2 [==============================] - 0s 9ms/step - loss: 41.7803 - mean_squared_error: 41.7803
Epoch 6/100
2/2 [==============================] - 0s 6ms/step - loss: 35.3312 - mean_squared_error: 35.3312
Epoch 7/100
2/2 [==============================] - 0s 8ms/step - loss: 23.3110 - mean_squared_error: 23.3110
Epoch 8/100
2/2 [==============================] - 0s 6ms/step - loss: 22.9722 - mean_squared_error: 22.9722
Epoch 9/100
2/2 [==============================] - 0s 8ms/step - loss: 28.6428 - mean_squared_error: 28.6428
Epoch 10/100
2/

In [39]:
# Now, we will predict the temperature for the next 3 days

# Predict the temperature for the next 3 days
predictions = model.predict(X_test_seq[-1].reshape(1, window_size))

print('Predicted temperature for the next 3 days:', predictions[0])

1/1 [==============================] - 0s 65ms/step
Predicted temperature for the next 3 days: [9.881827  9.3954735 7.9645042]
